In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import tensorflow_text as text
df_stock = pd.read_csv("icici/ICICIBANK.NS.csv",
                 parse_dates = ["Date"],
                 index_col = ["Date"])
df_posts = pd.read_excel("icici/icici_bank_final_posts.xlsx")
# removing the unnecessary columns
df_posts.drop(["Unnamed: 0"], axis=1,inplace=True)
# removing spam posts
df_posts = df_posts[df_posts.Spam==0.0]
df_posts.drop(["Spam"],axis=1,inplace=True)
# sliding a window of 7 days and adding all the TIs
from stock_helper import prepare_data
x,y = prepare_data(df_stock)
# slicing the data
final_x = x[np.datetime64("2021-11-13"):]
final_y = y[np.datetime64("2021-11-13"):]
# reversing the posts data
df_posts = df_posts[::-1]
final_posts = df_posts[7:]
# loading the sentiment analysis model
sent_model = tf.keras.models.load_model("final_bert")
# removing duplicates from the data
final_posts.drop_duplicates(subset=['Messages'])
# calculating the sentiments score
sentiments = []
prev = np.datetime64("2015-11-12 21:31:26")
for i in final_y.index:
    total=0
    cnt=0
    for j in final_posts.itertuples():
        _,msg,time = j
        if np.datetime64(time)<np.datetime64(i) and np.datetime64(time)>prev:
            total += tf.squeeze(sent_model.predict([msg])).numpy()
            cnt+=1
    prev = np.datetime64(i)
    if(cnt==0):
        sentiments.append(0)
    else:
        sentiments.append(total/cnt)
# getting indices where sentiments score is 0
zero_index = []
for i,j in enumerate(sentiments):
    if(j==0):
        zero_index.append(i)
# removing all the zero values indices
sentiments = np.delete(sentiments,zero_index)
final_x_zeros = final_x.copy()
final_y_zeros = final_y.copy()
final_y_zeros = final_y_zeros.to_frame()
final_x_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros = final_y_zeros[final_y_zeros.removal_assist.isin(zero_index)==False]
final_x_zeros = final_x_zeros[final_x_zeros.removal_assist.isin(zero_index)==False]
# removing the added helper column
final_x_zeros.drop(["removal_assist"], axis=1,inplace=True)
final_y_zeros.drop(["removal_assist"], axis=1, inplace=True)

In [2]:
tf.random.set_seed(42)
inputs = tf.keras.Input(shape=(71,1), name="inputs")
x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="causal")(inputs)
x = tf.keras.layers.MaxPooling1D()(x)
x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="causal")(x)
x = tf.keras.layers.MaxPooling1D()(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.models.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 71, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 71, 64)            256       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 35, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 35, 64)            12352     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 17, 64)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0     

In [3]:
model.compile(loss = "mae", optimizer=tf.keras.optimizers.Adam(), 
              metrics=["mae","mse"])
history = model.fit(final_x_zeros[:43],final_y_zeros[:43],
                    epochs = 200,
                    validation_data=(final_x_zeros[43:],final_y_zeros[43:]))

Epoch 1/200
2/2 [==============================] - 3s 247ms/step - loss: 828.4128 - mae: 828.4128 - mse: 688463.0625 - val_loss: 722.7546 - val_mae: 722.7546 - val_mse: 522476.8750
Epoch 2/200
2/2 [==============================] - 0s 23ms/step - loss: 662.6653 - mae: 662.6653 - mse: 441512.5938 - val_loss: 557.8418 - val_mae: 557.8418 - val_mse: 311285.6250
Epoch 3/200
2/2 [==============================] - 0s 24ms/step - loss: 507.0433 - mae: 507.0433 - mse: 259409.0156 - val_loss: 400.2975 - val_mae: 400.2975 - val_mse: 160335.0469
Epoch 4/200
2/2 [==============================] - 0s 23ms/step - loss: 358.1701 - mae: 358.1701 - mse: 129892.5000 - val_loss: 247.5738 - val_mae: 247.5738 - val_mse: 61390.0664
Epoch 5/200
2/2 [==============================] - 0s 24ms/step - loss: 212.8154 - mae: 212.8154 - mse: 46813.9258 - val_loss: 96.3811 - val_mae: 96.3811 - val_mse: 9390.6807
Epoch 6/200
2/2 [==============================] - 0s 23ms/step - loss: 72.0916 - mae: 72.0916 - mse: 645

2/2 [==============================] - 0s 22ms/step - loss: 21.4310 - mae: 21.4310 - mse: 626.8738 - val_loss: 16.8529 - val_mae: 16.8529 - val_mse: 362.8664
Epoch 49/200
2/2 [==============================] - 0s 23ms/step - loss: 23.8498 - mae: 23.8498 - mse: 859.7641 - val_loss: 13.5902 - val_mae: 13.5902 - val_mse: 237.1821
Epoch 50/200
2/2 [==============================] - 0s 23ms/step - loss: 21.5419 - mae: 21.5419 - mse: 660.7257 - val_loss: 10.4136 - val_mae: 10.4136 - val_mse: 207.7352
Epoch 51/200
2/2 [==============================] - 0s 26ms/step - loss: 22.1110 - mae: 22.1110 - mse: 637.7178 - val_loss: 15.8939 - val_mae: 15.8939 - val_mse: 403.0934
Epoch 52/200
2/2 [==============================] - 0s 24ms/step - loss: 22.2949 - mae: 22.2949 - mse: 664.8275 - val_loss: 11.5007 - val_mae: 11.5007 - val_mse: 249.2913
Epoch 53/200
2/2 [==============================] - 0s 24ms/step - loss: 21.1900 - mae: 21.1900 - mse: 548.4055 - val_loss: 11.3664 - val_mae: 11.3664 - val_m

Epoch 96/200
2/2 [==============================] - 0s 24ms/step - loss: 18.8403 - mae: 18.8403 - mse: 501.0805 - val_loss: 15.7378 - val_mae: 15.7378 - val_mse: 441.7759
Epoch 97/200
2/2 [==============================] - 0s 23ms/step - loss: 17.8697 - mae: 17.8697 - mse: 447.6784 - val_loss: 14.0385 - val_mae: 14.0385 - val_mse: 241.8929
Epoch 98/200
2/2 [==============================] - 0s 24ms/step - loss: 17.9432 - mae: 17.9432 - mse: 470.8245 - val_loss: 14.8803 - val_mae: 14.8803 - val_mse: 254.8666
Epoch 99/200
2/2 [==============================] - 0s 23ms/step - loss: 18.4171 - mae: 18.4171 - mse: 509.0380 - val_loss: 14.5461 - val_mae: 14.5461 - val_mse: 247.3487
Epoch 100/200
2/2 [==============================] - 0s 24ms/step - loss: 17.8815 - mae: 17.8815 - mse: 478.2939 - val_loss: 15.0160 - val_mae: 15.0160 - val_mse: 258.6189
Epoch 101/200
2/2 [==============================] - 0s 23ms/step - loss: 17.7392 - mae: 17.7392 - mse: 481.2440 - val_loss: 12.1564 - val_mae: 

Epoch 144/200
2/2 [==============================] - 0s 23ms/step - loss: 15.8762 - mae: 15.8762 - mse: 388.2766 - val_loss: 12.0805 - val_mae: 12.0805 - val_mse: 278.8721
Epoch 145/200
2/2 [==============================] - 0s 24ms/step - loss: 15.2805 - mae: 15.2805 - mse: 329.5874 - val_loss: 14.6405 - val_mae: 14.6405 - val_mse: 396.3718
Epoch 146/200
2/2 [==============================] - 0s 24ms/step - loss: 15.9406 - mae: 15.9406 - mse: 347.7302 - val_loss: 13.2603 - val_mae: 13.2603 - val_mse: 346.3864
Epoch 147/200
2/2 [==============================] - 0s 23ms/step - loss: 15.4848 - mae: 15.4848 - mse: 328.3020 - val_loss: 12.5564 - val_mae: 12.5564 - val_mse: 254.8651
Epoch 148/200
2/2 [==============================] - 0s 24ms/step - loss: 15.1355 - mae: 15.1355 - mse: 345.7443 - val_loss: 12.4458 - val_mae: 12.4458 - val_mse: 275.9726
Epoch 149/200
2/2 [==============================] - 0s 24ms/step - loss: 15.1566 - mae: 15.1566 - mse: 335.0591 - val_loss: 12.5678 - val_m

Epoch 192/200
2/2 [==============================] - 0s 23ms/step - loss: 15.5867 - mae: 15.5867 - mse: 331.8222 - val_loss: 14.1518 - val_mae: 14.1518 - val_mse: 367.1736
Epoch 193/200
2/2 [==============================] - 0s 25ms/step - loss: 14.8468 - mae: 14.8468 - mse: 314.7085 - val_loss: 14.8872 - val_mae: 14.8872 - val_mse: 399.7312
Epoch 194/200
2/2 [==============================] - 0s 24ms/step - loss: 14.3469 - mae: 14.3469 - mse: 303.1079 - val_loss: 13.6003 - val_mae: 13.6003 - val_mse: 235.6586
Epoch 195/200
2/2 [==============================] - 0s 23ms/step - loss: 15.9618 - mae: 15.9618 - mse: 383.3762 - val_loss: 14.3596 - val_mae: 14.3596 - val_mse: 248.0116
Epoch 196/200
2/2 [==============================] - 0s 24ms/step - loss: 16.7117 - mae: 16.7117 - mse: 415.6179 - val_loss: 13.5125 - val_mae: 13.5125 - val_mse: 350.2792
Epoch 197/200
2/2 [==============================] - 0s 23ms/step - loss: 14.1273 - mae: 14.1273 - mse: 304.4883 - val_loss: 17.0840 - val_m

In [4]:
model.evaluate(final_x_zeros[43:],final_y_zeros[43:])

1/1 [==============================] - 0s 24ms/step - loss: 16.6612 - mae: 16.6612 - mse: 479.5851


[16.661226272583008, 16.661226272583008, 479.5850830078125]

In [5]:
model.save("conv1d_icici")

INFO:tensorflow:Assets written to: conv1d_icici\assets


In [6]:
preds = tf.squeeze(model.predict(final_x_zeros[43:])).numpy()
preds

array([831.8659 , 829.67224, 826.1362 , 813.6705 , 814.49286, 813.731  ,
       815.881  , 815.4066 , 808.57196, 799.2481 , 797.8554 ],
      dtype=float32)